In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/MyDrive/Tooth_dataset"  # adjust if needed


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

dataset_path = "/content/drive/MyDrive/Tooth_dataset"

# Transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# Load dataset with transforms
full_dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)

# Split 80% train / 20% val
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply val transforms
val_dataset.dataset.transform = val_transform

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))


Training samples: 1126
Validation samples: 282


In [ ]:
from torchvision import models
import torch.nn as nn
import torch

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for 5 classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)  # 5 dental disease classes
model = model.to(device)

print(model)



/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)  # only train final layer


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

dataset_path = "/content/drive/MyDrive/Tooth_dataset"

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Load full dataset with train_transform
full_dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)

# Split 80% train / 20% val
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# For validation, wrap the subset with a dataset that applies val_transform
from torch.utils.data import Subset

class TransformedSubset(Subset):
    def __init__(self, subset, transform):
        super().__init__(subset.dataset, subset.indices)
        self.transform = transform
    def __getitem__(self, idx):
        x, y = super().__getitem__(idx)
        if self.transform:
            x = self.transform(x)
        return x, y

val_dataset = TransformedSubset(val_dataset, val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
from google.colab import files
files.download("/content/dental_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ----------------------------- Imports -----------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset
from tqdm.notebook import tqdm
import os

# ----------------------------- Paths -----------------------------
dataset_path = "/content/drive/MyDrive/Tooth_dataset"  # change if needed
save_path = "/content/dental_model.pth"

# ----------------------------- Transforms -----------------------------
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ----------------------------- Dataset -----------------------------
full_dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class TransformedSubset(Subset):
    def __init__(self, subset, transform):
        super().__init__(subset.dataset, subset.indices)
        self.transform = transform
    def __getitem__(self, idx):
        x, y = super().__getitem__(idx)
        if self.transform:
            x = self.transform(x)
        return x, y

val_dataset = TransformedSubset(val_dataset, val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ----------------------------- Device -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ----------------------------- Model -----------------------------
model = models.resnet18(pretrained=True)

# Freeze all layers except layer4 and fc
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Replace final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)  # 5 classes
model = model.to(device)

# ----------------------------- Loss + Optimizer -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# ----------------------------- Classes -----------------------------
classes = ['Calculus', 'Caries', 'Hypodontia', 'Mouth Ulcer', 'Tooth Discoloration']

# ----------------------------- Training Loop -----------------------------
num_epochs = 15
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Print batch loss every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f"Batch [{batch_idx+1}/{len(train_loader)}] - Loss: {loss.item():.4f}")

    train_loss = running_loss / len(train_loader)

    # ----------------- Validation -----------------
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}] - Avg Loss: {train_loss:.4f} - Val Accuracy: {val_accuracy:.2f}%")

    # Save checkpoint if validation improves
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save(model.state_dict(), save_path)
        print(f"Validation improved. Model saved at epoch {epoch+1} with Val Accuracy: {best_val_acc:.2f}%")

    scheduler.step()

print(f"Training complete. Best Val Accuracy: {best_val_acc:.2f}%")
print(f"Best model saved at: {save_path}")


Using device: cpu


Epoch 1/15:   0%|          | 0/71 [00:00<?, ?it/s]

Batch [10/71] - Loss: 1.3776
Batch [20/71] - Loss: 0.8845
Batch [30/71] - Loss: 0.4878
Batch [40/71] - Loss: 0.6510
Batch [50/71] - Loss: 0.7053
Batch [60/71] - Loss: 0.6072
Batch [70/71] - Loss: 0.3506
Epoch [1/15] - Avg Loss: 0.8357 - Val Accuracy: 81.91%
Validation improved. Model saved at epoch 1 with Val Accuracy: 81.91%


Epoch 2/15:   0%|          | 0/71 [00:00<?, ?it/s]

Batch [10/71] - Loss: 0.7433
Batch [20/71] - Loss: 0.5750
Batch [30/71] - Loss: 0.2538
Batch [40/71] - Loss: 0.3496
Batch [50/71] - Loss: 0.3650
Batch [60/71] - Loss: 0.1539
Batch [70/71] - Loss: 0.2110
Epoch [2/15] - Avg Loss: 0.3450 - Val Accuracy: 87.23%
Validation improved. Model saved at epoch 2 with Val Accuracy: 87.23%


...

Batch [10/71] - Loss: 0.0055
Batch [20/71] - Loss: 0.0219
Batch [30/71] - Loss: 0.0223
Batch [40/71] - Loss: 0.0070
Batch [50/71] - Loss: 0.0102
Batch [60/71] - Loss: 0.0500
Batch [70/71] - Loss: 0.0606
Epoch [13/15] - Avg Loss: 0.0734 - Val Accuracy: 92.55%


Epoch 14/15:   0%|          | 0/71 [00:00<?, ?it/s]

Batch [10/71] - Loss: 0.1350
Batch [20/71] - Loss: 0.0178
Batch [30/71] - Loss: 0.0305
Batch [40/71] - Loss: 0.0120
Batch [50/71] - Loss: 0.0214
Batch [60/71] - Loss: 0.0760
Batch [70/71] - Loss: 0.0654
Epoch [14/15] - Avg Loss: 0.0642 - Val Accuracy: 92.91%


Epoch 15/15:   0%|          | 0/71 [00:00<?, ?it/s]

Batch [10/71] - Loss: 0.1434
Batch [20/71] - Loss: 0.0270
Batch [30/71] - Loss: 0.1159
Batch [40/71] - Loss: 0.0777
Batch [50/71] - Loss: 0.0145
Batch [60/71] - Loss: 0.0400
Batch [70/71] - Loss: 0.0482
Epoch [15/15] - Avg Loss: 0.0549 - Val Accuracy: 91.13%
Training complete. Best Val Accuracy: 92.91%
Best model saved at: /content/dental_model.pth
